In [9]:
from datetime import timedelta, date, datetime
from tqdm import tqdm
import json
import asyncio
import aiohttp
from lxml.html import fromstring

def date_range(start_date, end_date):
    """
    find business days to parse
    :param start_date:
    :param end_date:
    :return: business days 
    """
    for n in range(int((end_date - start_date).days) + 1):
        if (start_date + timedelta(n)).weekday() not in (5, 6):
            yield start_date + timedelta(n)

def generate_links(dates, url):
    """generate links to parse from"""
    
    for date in dates:
        link = url.format(date)
        yield link

async def fetch(session, url):
    """download data from url and return it in json"""
    with aiohttp.Timeout(10):
        async with session.get(url) as response:
            html = await response.text()
            tree = fromstring(html)
            text = []
            
            for tr in tree.xpath('//tr'):
                text.append([x.strip().replace(u'\xa0', u'') for x in tr.text_content().split('\n')])
            
            names = [
                "not_used1",
                "Short",
                "ISIN",
                "Currency",
                "Open",
                "High",
                "Low",
                "Closing",
                "% daily change *",
                "Volume",
                "Number of trades",
                "Turnover (thous.)",
                "not_used2"]
            
            output = {}
            output[url[-10:]] = [dict(zip(names, t)) for indx, t in enumerate(text) if indx != 0]
            #return json.dumps(output, separators=(',', ':'))
            return output
        
        
async def fetch_all(loop, urls, sem):
    """task manager"""
    async with aiohttp.ClientSession(loop=loop) as session:
        tasks = []
        for url in tqdm(urls):
            await sem.acquire()
            task = asyncio.ensure_future(fetch(session, url))
            task.add_done_callback(lambda t: sem.release())
            task.add_done_callback(collector)
            task.add_done_callback(tasks.remove)
            tasks.append(task)

        results = await asyncio.gather(*tasks, return_exceptions=False)  # default is false, that would raise

    # for testing purposes only
    # gather returns results in the order of coros, works if urls is a list
    [print('ERROR: ', url) for url in results if isinstance(url, BaseException)]    
    return results
    
    
# TODO queue???
collector_data = {}
def collector(cdata):
    global collector_data
    collector_data.update(cdata.result())

    if len(collector_data) <= 10:
        #print(len(collector_data))
        collector_data.append(cdata.result())
    else:
        #print('save')
        save_to_file(collector_data)
        collector_data.clear()
        #print(len(collector_data))
        collector_data.append(cdata.result())

        
def save_to_file(data_save):
    """save data to filename"""
    # todo change save to separate file -> queue or redis 
    with open("{}.txt".format(datetime.now().strftime('%Y%m%d%H%M%S%f')), "w") as f:
        data = json.dumps(data_save, separators=(',', ':'))
        # todo string to normal json
        f.write(data[1:-1] + ',')

        
if __name__ == '__main__':

    start = date(1997, 12, 10)
    end = date(2016, 10, 18)
    # index
    #url = 'https://www.gpw.pl/notowania_archiwalne_full_en?type=1&date={}'
    # futures
    url = 'https://www.gpw.pl/notowania_archiwalne_full_en?type=35&date={}'
    
    headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.6,en;q=0.4',
    'Upgrade-Insecure-Requests': 1,
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36',
    'Referer': 'https://www.gpw.pl/notowania_archiwalne_en?type=1&date=2016-10-12&show.x=32&show.y=12',
    'Cookie': 'PHPSESSID=9ms5i5ot338lkmorctqp7uf0o4; SID=!Khgk9sVQQ1zJmCEx3T9eKbrcTnC4jFE0LGIk/49Mys+u4PdrgyyJb0IcJBmyj7v7VF/taLxvsP5lqyaqEOoMZ6TrVZn9eTFrkLPO6nsSigfv2CbqJEbbVNDh9ZvRKVhSC6KG8zuzU3TIMe/hWIBqmwKcELYW7uw=; lang_code=EN; TS014040d7=016c1ed7ff34b3ee8cd0041f3724d83f5f0b455c8c9163223d541f4b3fbd0dfaf0f8ef6af01d96c806f14c8fd0f025c4cc8135b8ca32fae5a197c21b401a054bd3992bbdedcd89131a55b62bb9536890c98fda94c7',
    'Connection': 'keep-alive'}
    
    loop = asyncio.get_event_loop()
    sem = asyncio.Semaphore(5, loop=loop)
    urls = generate_links(date_range(start, end), url)
    #future = asyncio.ensure_future(fetch_all(loop, urls, sem))
    loop.run_until_complete(asyncio.gather(fetch_all(loop, urls, sem)))
    loop.close()

4920it [24:33,  2.13it/s]


TimeoutError: 

In [ ]:
"""соединяем файлы в один"""
import os
import json
from collections import OrderedDict


file_names = []
data = {}

for d, dirs, files in os.walk('C:\\Users'):
    file_names = files
    break
print(file_names)
for file in file_names:
    if file.startswith('2016'):
        with open(file) as f:
            content = f.readlines()
            js_data = json.loads('{' + content[0][:-1] + '}')
            data.update(js_data)

order_d = OrderedDict()
for i in sorted(data):
    order_d.update({i: data[i]})

with open('_futures2.txt', 'w') as f:
    _data = json.dumps(order_d, separators=(',', ':'))
    f.write(_data)


In [10]:
import json
from collections import OrderedDict


with open('_futures2.txt') as f1, open('_futures1.txt') as f2:
    l1 = f1.readlines()
    l2 = f2.readlines()

    d1 = json.loads(l1[0])
    d2 = json.loads(l2[0])


print(len(d1), len(d2))

c = {}
c.update(d1)
c.update(d2)


print(len(c))



full = OrderedDict()

for i in sorted(c):
    full.update({i: c[i]})

print(len(full))

with open('all_futs.txt', 'w') as a:
    _data = json.dumps(full, separators=(',', ':'))
    a.write(_data)

4264 3952
4761
4761
